In [1]:
## US Department of Transportation flight data
import pandas as pd

In [2]:
# Download the data file first:
# https://github.com/multidis/hult-sf-bus-challenge/raw/refs/heads/main/data_collection/bts/T_T100_MARKET_ALL_CARRIER.csv
# save it locally. The data covers 2024-01 through 2024-07.
# For a wider time range:
# https://www.transtats.bts.gov/DL_SelectFields.aspx
fcsv = "T_T100_MARKET_ALL_CARRIER.csv"
df = pd.read_csv(fcsv).rename(lambda x: x.lower(), axis=1)

In [8]:
df.head()

,passengers,freight,mail,distance,unique_carrier,unique_carrier_name,origin_airport_id,origin,origin_city_name,origin_state_abr,...,dest_city_name,dest_state_abr,dest_state_nm,dest_country,dest_country_name,year,quarter,month,distance_group,class
0,0.0,0.0,0.0,0.0,27Q,"Jet Aviation Flight Services, Inc.",15167,TEB,"Teterboro, NJ",NJ,...,"Teterboro, NJ",NJ,New Jersey,US,United States,2024,3,7,1,L
1,0.0,0.0,0.0,0.0,27Q,"Jet Aviation Flight Services, Inc.",15447,UGN,"Waukegan, IL",IL,...,"Waukegan, IL",IL,Illinois,US,United States,2024,2,4,1,L
2,0.0,0.0,0.0,0.0,2EQ,"J&M Alaska Air Tours, Inc. d/b/a Alaska Air Tr...",13467,MRI,"Anchorage, AK",AK,...,"Anchorage, AK",AK,Alaska,US,United States,2024,2,5,1,F
3,0.0,0.0,0.0,0.0,2LQ,Tropic Ocean Airways LLC,11697,FLL,"Fort Lauderdale, FL",FL,...,"Fort Lauderdale, FL",FL,Florida,US,United States,2024,2,6,1,L
4,0.0,0.0,0.0,0.0,2LQ,Tropic Ocean Airways LLC,16541,OBE,"Okeechobee, FL",FL,...,"Okeechobee, FL",FL,Florida,US,United States,2024,2,6,1,L


In [9]:
# top carries flying the largest number of passengers out of the U.S.
top_by_passengers = df\
    .query('origin_country_name == "United States" and dest_country_name != "United States"')\
    .groupby('unique_carrier_name').passengers.sum().nlargest(10)
top_by_passengers

unique_carrier_name
American Airlines Inc.                                         10728346.0
United Air Lines Inc.                                          10181688.0
Delta Air Lines Inc.                                            7988790.0
JetBlue Airways                                                 3277862.0
British Airways Plc                                             2217848.0
Air Canada                                                      2009363.0
Westjet                                                         1665980.0
Concesionaria Vuela Compania De Aviacion SA de CV (Volaris)     1623268.0
Southwest Airlines Co.                                          1489668.0
Lufthansa German Airlines                                       1457575.0
Name: passengers, dtype: float64

In [45]:
da=df[(df['origin_country_name']=='United States') & (df['dest_country_name']!='United States')]
da =da[['unique_carrier_name','passengers']].groupby(['unique_carrier_name']).sum().sort_values(by=['passengers'],ascending=False)
da.head(10)

,passengers
unique_carrier_name,
American Airlines Inc.,10728346.0
United Air Lines Inc.,10181688.0
Delta Air Lines Inc.,7988790.0
JetBlue Airways,3277862.0
British Airways Plc,2217848.0
Air Canada,2009363.0
Westjet,1665980.0
Concesionaria Vuela Compania De Aviacion SA de CV (Volaris),1623268.0
Southwest Airlines Co.,1489668.0


In [5]:
# most popular destination country for each carrier
df_dest = df.query('origin_country_name == "United States" and dest_country_name != "United States"')\
    .groupby(['unique_carrier_name', 'dest_country_name']).passengers.sum()\
    .groupby(level=0, group_keys=False).nlargest(1)
df_dest

unique_carrier_name                  dest_country_name
1263343 Alberta Inc. d/b/a Lynx Air  Canada                 27253.0
1903 Aviation                        Portugal                   5.0
ABS JETS A.S                         Czech Republic             4.0
ABSA-Aerolinhas Brasileiras          Argentina                  0.0
ABX Air Inc                          Belgium                    0.0
                                                            ...    
Western Global                       Bahrain                    0.0
Westjet                              Canada               1665980.0
World2Fly Portugal SA                Spain                    263.0
Xiamen Airlines Co., Ltd.            China                  34272.0
ZIPAIR Tokyo Inc.                    Japan                 158149.0
Name: passengers, Length: 241, dtype: float64

In [6]:
# most popular destinations for each top carrier specifically
df_dest[top_by_passengers.index]

unique_carrier_name                                          dest_country_name 
American Airlines Inc.                                       Mexico                2420906.0
United Air Lines Inc.                                        Mexico                1815916.0
Delta Air Lines Inc.                                         Mexico                1365350.0
JetBlue Airways                                              Dominican Republic    1076404.0
British Airways Plc                                          United Kingdom        2217638.0
Air Canada                                                   Canada                2009363.0
Westjet                                                      Canada                1665980.0
Concesionaria Vuela Compania De Aviacion SA de CV (Volaris)  Mexico                1623268.0
Southwest Airlines Co.                                       Mexico                 842432.0
Lufthansa German Airlines                                    Germany               

In [55]:
da = df[(df['origin_country_name'] == 'United States') & (df['dest_country_name'] != 'United States')]
da = da[['unique_carrier_name', 'dest_country_name', 'passengers']].groupby(['unique_carrier_name', 'dest_country_name']).sum().sort_values(['passengers'], ascending=False)
sorted = da.groupby('unique_carrier_name', group_keys=False).apply(lambda x: x.nlargest(1, 'passengers')).sort_values(['passengers'], ascending=False)
sorted = sorted.reset_index()  # Fix alignment issue
sorted.head(10)

,unique_carrier_name,dest_country_name,passengers
0,American Airlines Inc.,Mexico,2420906.0
1,British Airways Plc,United Kingdom,2217638.0
2,Air Canada,Canada,2009363.0
3,United Air Lines Inc.,Mexico,1815916.0
4,Westjet,Canada,1665980.0
5,Concesionaria Vuela Compania De Aviacion SA de...,Mexico,1623268.0
6,Lufthansa German Airlines,Germany,1457575.0
7,Delta Air Lines Inc.,Mexico,1365350.0
8,Compagnie Natl Air France,France,1244634.0
9,JetBlue Airways,Dominican Republic,1076404.0


In [57]:
## the queries above were adopted from the following tutorial:
## https://github.com/stefmolin/pandas-workshop/blob/main/asynchronous_lab/solutions.ipynb
## Find more useful examples in there by Stefanie Molin, https://github.com/stefmolin

%pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=420e3ec1e4bccb9299ad07e77d94e2b78420d8fb54f3eead5fbeaec20b16e337
  Stored in directory: /Users/jorgeluissolis/Library/Caches/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [58]:
## if you prefer the SQL-way:
## conda install -c conda-forge pandasql
from pandasql import sqldf

In [59]:
query = """
  SELECT * 
  FROM df
  WHERE origin_country_name == "United States"
  LIMIT 5;
"""

In [60]:
dfsel = sqldf(query, locals())
dfsel

,passengers,freight,mail,distance,unique_carrier,unique_carrier_name,origin_airport_id,origin,origin_city_name,origin_state_abr,...,dest_city_name,dest_state_abr,dest_state_nm,dest_country,dest_country_name,year,quarter,month,distance_group,class
0,0.0,0.0,0.0,0.0,27Q,"Jet Aviation Flight Services, Inc.",15167,TEB,"Teterboro, NJ",NJ,...,"Teterboro, NJ",NJ,New Jersey,US,United States,2024,3,7,1,L
1,0.0,0.0,0.0,0.0,27Q,"Jet Aviation Flight Services, Inc.",15447,UGN,"Waukegan, IL",IL,...,"Waukegan, IL",IL,Illinois,US,United States,2024,2,4,1,L
2,0.0,0.0,0.0,0.0,2EQ,"J&M Alaska Air Tours, Inc. d/b/a Alaska Air Tr...",13467,MRI,"Anchorage, AK",AK,...,"Anchorage, AK",AK,Alaska,US,United States,2024,2,5,1,F
3,0.0,0.0,0.0,0.0,2LQ,Tropic Ocean Airways LLC,11697,FLL,"Fort Lauderdale, FL",FL,...,"Fort Lauderdale, FL",FL,Florida,US,United States,2024,2,6,1,L
4,0.0,0.0,0.0,0.0,2LQ,Tropic Ocean Airways LLC,16541,OBE,"Okeechobee, FL",FL,...,"Okeechobee, FL",FL,Florida,US,United States,2024,2,6,1,L


In [11]:
## if you prefer the SQL way to begin with: example tutirial with the same type of data
## https://github.com/dannguyen/bts-transstats-t100-domestic-demo